In [130]:
import os
import os.path as osp
import numpy as np
import cv2
import matplotlib.pyplot as plt

from ImageFeatureExtraction.kernel import gaussian_kernel
from ImageFeatureExtraction.operation import Conv2D

In [2]:
def imcompare(imgs:list):
    plt.figure(dpi=500)
    plt.imshow(np.hstack(imgs), cmap='gray')
    plt.show()

In [3]:
testimg = cv2.imread(osp.join("demo", "dog.jpg"), 0).astype(np.float32)
testimg.shape

(608, 1080)

In [108]:
def DoG_pyramid(img:np.ndarray, scale_num=5,sigma:float=1.6)->np.ndarray:
    k = 2**(1/(scale_num-1))
    kernels = gaussian_kernel(ksize=7, sigma=[sigma*(k**i) for i in range(scale_num)])
    doglist = [Conv2D(img, kernels[i+1]-kernels[i]) for i in range(len(kernels)-1)]
    return np.dstack(doglist)



In [123]:
def Scale_Space_Extrema_Detection(cmp:np.ndarray)->np.ndarray:
    extrema_idx = np.zeros((cmp.shape[0],cmp.shape[1]),dtype=np.int32)
    for k in range(1, cmp.shape[2]-1):
        for i in range(cmp.shape[0]):
            x_start = i-1 if i-1> 0 else 0
            for j in range(cmp.shape[1]):
                y_start = j-1 if j-1 > 0 else 0
                cube = cmp[x_start:i+2, y_start:j+2, k-1:k+2].transpose((2,0,1))
                cx, cy, cz = 1,1 if i-1 >= 0 else 0, 1 if j-1 >= 0 else 0
                center = cube[cx, cy, cz]
                rmax= (center >= cube).astype(np.int32)
                if np.sum(rmax) == cube.size:
                    extrema_idx[i,j] = 1
                    continue
                rmin = (center <= cube).astype(np.int32)
                if np.sum(rmin) == cube.size :
                    extrema_idx[i,j] = 1
    return extrema_idx
    

In [124]:
K = Scale_Space_Extrema_Detection(DoG_pyramid(img=testimg))

1
2


In [125]:
see = testimg*(1+K*255)
